In [1]:
%cd efficientnet_keras_transfer_learning/

D:\AAIC\cs2\efficientnet_keras_transfer_learning


In [2]:
from efficientnet import EfficientNetB3 as ENet3b
%cd ../

Using TensorFlow backend.


D:\AAIC\cs2


In [3]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import  Model
from tensorflow.keras.optimizers import RMSprop, Adam

In [4]:
img_width, img_height = 300,300
batch_size = 8

model = ENet3b(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3), pooling='avg')


x = model.output
x = BatchNormalization()(x)
x = Dropout(0.75)(x)
x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
predictions = Dense(16, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer= Adam(0.0003), metrics=["accuracy"])

model_final.load_weights('HDF5/DocClassification_TL-B3_07.hdf5')

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [5]:
from os import path
from keras.preprocessing.image import ImageDataGenerator
def final_model(folder, stps):
    """Given a path to a document drectory, returns accuracy.
    path: path to directory where documents are arranged in 16 folders named
          0 to 15 based on category
    """
    datagen = ImageDataGenerator(rescale=1. / 255)
    
    img_generator = datagen.flow_from_directory(
                folder,
                target_size=(img_width, img_height),
                color_mode='rgb',
                batch_size=64,
                class_mode='categorical')
    
    pred = model_final.evaluate_generator(img_generator,steps=stps//64,verbose=0)
    
    return pred[1]*100

In [6]:
train_acc=final_model('traindata/',319835)
val_acc=final_model('valdata/',39995)
test_acc=final_model('testdata/', 39995)


Found 319835 images belonging to 16 classes.
Found 39995 images belonging to 16 classes.
Found 39996 images belonging to 16 classes.


In [7]:
print("Accuracy on train data: {0:.2f}%".format(train_acc))
print("Accuracy on validation data: {0:.2f}%".format(val_acc))
print("Accuracy on test data: {0:.2f}%".format(test_acc))

Accuracy on train data: 95.44%
Accuracy on validation data: 91.40%
Accuracy on test data: 91.17%
